In [10]:
%%writefile us_covid_data_crawler.py
# 미국의 주 명(State Name)을 입력하면 미국 전반과 해당 주의 COVID-19 인구통계 정보를 제공해주는 프로그램

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import random
import time
import datetime
from urllib.request import Request, urlopen

class Program_Starter:
    states_full_name = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
                            "Connecticut", "Delaware", "District of Columbia", "Florida", "Georgia", 
                            "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky",
                            "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
                            "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire",
                            "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota",
                            "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island",
                            "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah",
                            "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
        
    states_abbreviated = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "FL", "GA", "HI",
                              "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN",
                              "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH",
                              "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA",
                              "WV", "WI", "WY"]
        
    states_dictionary = dict(zip(states_abbreviated, states_full_name))
    
    def __init__(self):
        print("******************************************************")
        print("***      WELCOME TO US COVID-19 DATA TRACKER.      ***")
        print("******************************************************\n")
        print("THIS PROGAMS OFFERS THE INFORMATION REGARDING UNITED STATES OF AMERICA CORONAVIRUS(COVID-19) STATISTICS.")
        print("DATA SOURCES ARE CRALWED FROM WORLDOMETERS.COM THAT ARE BASED ON AGGREGATED COUNTS REPORTED BY US GOVERNMENT.")
    
        print("\nUS COVID-19 DATA TRACKER WILL OFFER...")
        print("1. US VISUALIZED COVID DATA")
        print("2. STATE VISUALIZED COVID DATA\n")
        
        now_time = datetime.datetime.today()
        print("@@ Current Program Launch point in time: {}\n".format(now_time.strftime('%Y-%m-%d %H:%M:%S %a')))

    def showStateName(self):
        # 주의 이름(full name)과 약칭(abbreviated)을 소개해주는 함수
        print("Choose one of state that you want to investigate among the below lists.\n")
           
        states_DataFrame = list(zip(Program_Starter.states_full_name, Program_Starter.states_abbreviated))
        states_DataFrame = pd.DataFrame(data = states_DataFrame, columns = ["State Name", "Abb. Name"])
        print(states_DataFrame)
        
    def inputStateName():
        # 예외처리(Exception Handling)을 활용하여 주의 이름 입력받기 (1)
        # 주의 이름(states_full_name) 및 약어 이름(states_abbreviated) 리스트에 속하지 않을 경우 오류 raise
        StateName = input("Enter the name of the state that you would like to take a look at: ")
        
        if (StateName.title() in Program_Starter.states_full_name) or (StateName.upper() in Program_Starter.states_abbreviated):
            return StateName
        else:
            raise NameException("PLEASE TYPE CORRECT STATES NAME!\n")     

    def getStateName(self):
        # 예외처리(Exception Handling)을 활용하여 주의 이름 입력받기 (2)
        # 예외 발생시 정확한 주 명을 입력할 때까지 while loop 반복
        while(True):
            try:
                StateName = Program_Starter.inputStateName()
                
            except NameException as e:
                print("\n!! WARNING !!")
                print("You typed in the invalid states name.\n")
                print(e.args[0])
                
            else:
                if StateName.title() in Program_Starter.states_full_name:
                    print("\n\nSUCCEED! Requested data on <%s> will be loaded."%StateName.title())
                    StateName = StateName.title()
                    return StateName
        
                elif (StateName.upper() in Program_Starter.states_abbreviated):
                    print("\n\nSUCCEED! Requested data on <%s> will be loaded."%Program_Starter.states_dictionary[StateName.upper()])
                    StateName = Program_Starter.states_dictionary[StateName.upper()]
                    return StateName


class DataCrawler(Program_Starter):
    # 데이터 크롤링을 주관하는 클래스 (Program_Starter을 상속)
    def __init__(self, data):
        self.data = data
        # data는 StateName에 해당

    def DataCrawling(self):
        start = time.time()
        print("PLEASE WAIT FOR A SECOND FOR GETTING YOUR REQUEST DATA ON-", self.data, ".....")
        # worldometers.com 에서 데이터 불러오기
        req = Request("https://www.worldometers.info/coronavirus/country/us/", headers={"User-Agent": "Mozilla/5.0"})
        worldometers_web = urlopen(req).read()
        data_tables = pd.read_html(worldometers_web)
        COVIDdata = data_tables[0]
        # 데이터프레임에서 불필요한 행과 열 제거
        COVIDdata = COVIDdata.drop(0)
        COVIDdata = COVIDdata.drop(range(52,65))
        COVIDdata.drop(['#', 'Source', 'Projections'], axis = 1, inplace = True)
        # 데이터프레임의 열 명(column name) 정리
        COVIDdata.columns = ['USAState', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'ActiveCases', 'TotCases1Mpop', 'Deaths1Mpop',
       'TotalTests', 'Tests1Mpop', 'Population']
        
        print("\nSUCCESSFULLY CRAWLLED DATA FOR US COVID-19 DATA TRACKER!")
        # 실행시간 점검 (데이터 크롤링에 소요된 시간)
        end = time.time()
        print("\DATA CRAWLING TOTAL RUNTIME:", end - start)
        
        return COVIDdata

    
class NameException(Exception):
    # 예외처리(Exception Handling)를 담당하는 클래스
    def __init__(self, message):
        self._message = message
        
        
class UScovid19:
    # 미국 전반의 COVID-19 상황을 제시하는 클래스
    def __init__(self, StateName, COVIDdata):
        self.StateName = StateName        
        self.COVIDdata = COVIDdata
        print("******************************************************")
        print("**            US COVID-19 DATA TRACKER!             **")
        print("******************************************************\n")
        print("Below is the overview of the current situation of the United States!\n")
        print(COVIDdata[["USAState", "TotalCases", "TotalDeaths", "TotalRecovered", "ActiveCases"]])
        
    @staticmethod
    def UStotalCases(self):
        # 미국 전역의 총 확진 정보에 대한 시각화 그래프
        # 사망자 수, 회복자 수, 확진자 수에 대한 그래프 제공
        plt.rcParams["figure.figsize"] = (15, 10)
        visualized_cases = self.COVIDdata.sort_values(by = "TotalCases", ascending = False).head(10)
        visualized_cases = visualized_cases.set_index("USAState")
        visualized_cases = visualized_cases[["TotalCases", "TotalDeaths", "TotalRecovered", "ActiveCases"]]
        
        bar = visualized_cases.plot.bar(grid = True)   
        plt.title("US COVID-19 TOP10 TOTAL CASES", fontsize = 25)
        plt.xlabel("USA states", fontsize = 15); plt.ylabel("Number (1million)", fontsize = 15)
        plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
        plt.show()
    
    @staticmethod
    def UStotalTests(self):
        # 미국 전역의 총 검사 수에 대한 시각화 그래프
        visualized_deaths = self.COVIDdata.sort_values(by = "TotalTests", ascending = False).head(15)
        c = random.choices(list(mcolors.CSS4_COLORS.values()),k = len(visualized_deaths.index))
        plt.figure(figsize = (20, 15))
        plt.title("Covid-19 Total Tests Counts", size = 20)
        plt.pie(visualized_deaths["TotalTests"], colors = c, autopct = "%.1f%%")
        plt.legend(visualized_deaths["USAState"], loc = "best", fontsize = 15)
        plt.show()

        
class CountryCOVID(UScovid19):
    # UScovid19 에서 설정한 미국 전역의 코로나 상황 데이터를 상속받은 클래스
    # 세부 코로나 상황에 대한 시각화 정보를 제공하는 클래스
    def __init__(self, COVIDdata):
        self.COVIDdata = COVIDdata
        
    def TotalCases(self):   # TCV : Total Cases Visualization
        print("\t<Total Confirmed Cases> of the USA")
        # 미국 내 총 감염자 수에 대한 그래프
        TCV = self.COVIDdata.head(20)    
        plt.figure(figsize = (10, 10))
        plt.barh(TCV["USAState"], TCV["TotalCases"], color = "maroon")
        plt.title("US states with the Most Total Confirmed Cases", fontsize = 30)
        plt.xlabel("Total Confirmed Cases", fontsize = 25); plt.ylabel("States", fontsize = 25)
        plt.xticks(fontsize = 15); plt.yticks(fontsize = 25)
        plt.show()
    
    def TotalDeaths(self):   # TDV : Total Deaths Visualization
        print("\t<Total Deaths Cases> of the USA")
        # 미국 내 총 사망자 수에 대한 그래프
        TDV = self.COVIDdata.sort_values(by = "TotalDeaths", ascending = False).head(10)
        plt.figure(figsize = (15, 10))
        plt.bar(TDV["USAState"], TDV["TotalDeaths"], color = "darkgreen")
        plt.title("US states with the Most Total Deaths Cases", fontsize = 30)
        plt.xlabel("Total Death Cases", fontsize = 25); plt.ylabel("States", fontsize = 25)
        plt.xticks(fontsize = 15); plt.yticks(fontsize = 25)
        plt.show()
    
    def TotalRecovered(self):   # TRV : Total Recovered Visualization
        print("\t<Total Recovered Cases> of the USA")
        # 미국 내 총 회복자 수에 대한 그래프
        # 전체 확진자 정보는 선 그래프로 추가하여 비교 분석 가능
        TRV = self.COVIDdata.sort_values(by = "TotalRecovered", ascending = False).head(10)
        fig, ax1 = plt.subplots(figsize = (17, 7)); my_color = 'tab:red'
        
        ax1 = sns.barplot(x = "USAState", y = "TotalRecovered", data = TRV, palette = "summer")
        ax1.tick_params(axis = "y")
        ax2 = sns.set(style = "ticks", rc = {"lines.linewidth" : 4.1}); ax2 = ax1.twinx()
        ax2.set_ylabel('Avg Percipitation %', fontsize = 12)
        ax2 = sns.lineplot(x = "USAState", y = "TotalCases", data = TRV, color = my_color)
        ax2.tick_params(axis = "y", color = my_color)
        plt.show()

    def TotalActiveCases(self):   # TAV : Total Active Cases Visualization
        print("\t<Total Active Cases> of the USA")
        # 미국 내 총 확진자 중 회복하지 않은 감염자 수에 대한 그래프
        TAV = self.COVIDdata.sort_values(by = "ActiveCases", ascending = False).head(10)
        plt.figure(figsize = (15, 10))
        plt.pie(TAV["ActiveCases"], labels = TAV["USAState"], shadow = True, startangle = 90, autopct = "%.1f%%")
        plt.show()
        
    

class StateCOVID(UScovid19):
    # 프로그램 초반부에서 입력받은 주(state)의 세부적인 코로나 상황에 대한 그래프 제공
    # UScovid19 클래스를 상속받아서 주 별 특징적인 정보를 제공해주는 클래스
    def __init__(self, StateName, COVIDdata):
        self.StateName = StateName
        self.COVIDdata = COVIDdata
        print("******************************************************")
        print("**            US COVID-19 DATA TRACKER!             **")
        print("******************************************************\n")
        print("REQUESTED STATE NAME ::: ", self.StateName)
        # 데이터프레임의 index name을 주 명(State Name)으로 변경
        state_data = self.COVIDdata.set_index("USAState")
        # 요청받은 주의 기본 코로나 상황 정보 제공
        print(state_data.loc[self.StateName])
         

    def tests_information(self):  
        # 주의 검사 현황에 대한 그래프 제공
        # 총 검사자 수 vs 검사받지 않은 사람의 수
        state_datum = self.COVIDdata.set_index("USAState")
        state_datum = dict(state_datum.loc[self.StateName])
        state_data_col = list(state_datum.keys())
        state_data_val = list(state_datum.values())
           
        data = [state_data_val[8], state_data_val[10] - state_data_val[8]]
        data_label = ["Total Tests", "Not Tested"]
        plt.title("Total Tests among State Population", size = 25)
        plt.pie(data, labels = data_label, autopct = "%.1f%%")
        plt.legend(data_label, loc = "best", fontsize = 15)
        plt.show()
        
    def confirmed_information(self):
        # 주의 코로나 발생 전반의 현황
        # 확진자 수, 완치자 수, 현재 감염자 수, 사망자 수에 대한 그래프 제공
        state_datum = self.COVIDdata.set_index("USAState")
        state_datum = dict(state_datum.loc[self.StateName])
        state_data_col = list(state_datum.keys())
        state_data_val = list(state_datum.values())

        setdata = [state_data_col[0], state_data_col[2], state_data_col[4], state_data_col[5]]
        namedata = [state_data_val[0], state_data_val[2], state_data_val[4], state_data_val[5]]
        plt.barh(setdata, namedata,  color = "deepskyblue")
        plt.title("COVID-19 CASES of {}".format(self.StateName), fontsize = 25)
        plt.xlabel("{}".format(self.StateName), fontsize = 25)
        plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
        plt.show()
    
    def per_information(self):
        # 주의 코로나 발생 전반의 현황
        # 인구 1백만명(million) 당(per) 확산 내용
        state_datum = self.COVIDdata.set_index("USAState")
        state_datum = dict(state_datum.loc[self.StateName])
        state_data_col = list(state_datum.keys())
        state_data_val = list(state_datum.values())

        setdata = [state_data_col[6], state_data_col[7], state_data_col[9]]
        namedata = [state_data_val[6], state_data_val[7], state_data_val[9]]
        plt.barh(setdata, namedata,  color = "lightcoral")
        plt.title("COVID-19 CASES of {}".format(self.StateName), fontsize = 25)
        plt.xlabel("{}".format(self.StateName), fontsize = 25)
        plt.xticks(fontsize = 20); plt.yticks(fontsize = 20)
        plt.show()
        

if __name__ == "__main__":
    myProgram = Program_Starter()
    # 미국 주의 이름 목록 제시
    myProgram.showStateName()
    # 사용자가 조회를 희망하는 주의 이름 입력
    StateName = myProgram.getStateName()
    # 데이터 크롤링
    COVIDdata = DataCrawler(StateName).DataCrawling()
    # 미국 코로나 상황에 대한 전반적인 소개
    us_covid_data = UScovid19(StateName, COVIDdata)
    UScovid19.UStotalCases(us_covid_data)
    UScovid19.UStotalTests(us_covid_data)
    whole_country_data = CountryCOVID(COVIDdata)
    whole_country_data.TotalCases()
    whole_country_data.TotalDeaths()
    whole_country_data.TotalRecovered()
    whole_country_data.TotalActiveCases()
    # 입력한 주의 코로나 상황에 대한 전반적인 소개
    myStatedata = StateCOVID(StateName, COVIDdata)
    StateCOVID.tests_information(myStatedata)
    StateCOVID.confirmed_information(myStatedata)
    StateCOVID.per_information(myStatedata)

Overwriting us_covid_data_crawler.py


### 전체 프로그램 실행

In [ ]:
%run us_covid_data_crawler.py

### 개별 실행

In [ ]:
myProgram = Program_Starter()

In [ ]:
myProgram.showStateName()

In [ ]:
StateName = myProgram.getStateName()

In [ ]:
COVIDdata = DataCrawler(StateName).DataCrawling()

In [ ]:
us_covid_data = UScovid19(StateName, COVIDdata)

In [ ]:
UScovid19.UStotalCases(us_covid_data)

In [ ]:
UScovid19.UStotalTests(us_covid_data)

In [ ]:
whole_country_data = CountryCOVID(COVIDdata)
whole_country_data.TotalCases()

In [ ]:
whole_country_data.TotalDeaths()

In [ ]:
whole_country_data.TotalRecovered()

In [ ]:
whole_country_data.TotalActiveCases()

In [ ]:
myStatedata = StateCOVID(StateName, COVIDdata)

In [ ]:
StateCOVID.tests_information(myStatedata)

In [ ]:
StateCOVID.confirmed_information(myStatedata)

In [ ]:
StateCOVID.per_information(myStatedata)